# Práctica: El Problema del Bandido Multibrazo (K-Armed Bandit)

<a href="https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno es el punto de entrada principal para la práctica. Desde aquí se configura el entorno de trabajo y se accede a los estudios detallados de cada algoritmo implementado.

In [1]:
import os
import sys

# --- CONFIGURACIÓN DEL REPOSITORIO ---
USER = "kikaymusic"
REPO = "EscuderoRodriguezSanchez"
BRANCH = "dev"
FOLDER = "k_brazos" # Carpeta donde están los archivos

URL_REPO = f"https://github.com/{USER}/{REPO}.git"

# Clonar el repositorio
if not os.path.exists(REPO):
    print(f"Clonando el repositorio {REPO}...")
    !git clone -b {BRANCH} {URL_REPO}
else:
    print("El repositorio ya está clonado.")

# Ajustar el Path

repo_path = os.path.abspath(os.path.join(REPO, FOLDER))
if repo_path not in sys.path:
    sys.path.append(repo_path)
    print(f"Directorio de trabajo añadido al path: {repo_path}")

# Moverse al directorio para que las rutas relativas funcionen
os.chdir(repo_path)

print(f" Entorno configurado. Directorio actual: {os.getcwd()}")

Clonando el repositorio EleazarRodriguezSanchez...
Cloning into 'EleazarRodriguezSanchez'...
fatal: could not read Username for 'https://github.com': No such device or address
Añadido /content/EleazarRodriguezSanchez al path del sistema.
Entorno configurado correctamente.


## Introducción al Problema

El objetivo de esta práctica es analizar el comportamiento de distintos algoritmos de aprendizaje por refuerzo ante el dilema de **exploración vs explotación**. Se utilizan "brazos" con diferentes distribuciones de probabilidad (Normal, Binomial y Bernoulli) para evaluar la capacidad de cada agente para maximizar la recompensa acumulada.

Se han implementado tres estrategias principales basándose en la literatura clásica de Sutton y Barto:
1.  **Epsilon-Greedy**
2.  **UCB1**
3.  **Softmax**

## Navegación entre Estudios

Haz clic en los siguientes enlaces para abrir los cuadernos de análisis específicos en Google Colab:

* ### [1. Experimento Epsilon-Greedy](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_epsilon_greedy.ipynb)
    * Análisis de la influencia del parámetro de exploración aleatoria.
* ### [2. Experimento UCB1](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_ucb.ipynb)
    * Estudio del optimismo ante la incertidumbre mediante cotas de confianza.
* ### [3. Experimento Softmax](https://colab.research.google.com/github/kikaymusic/EscuderoRodriguezSanchez/blob/dev/k_brazos/bandido_softmax.ipynb)
    * Evaluación de la selección probabilística basada en la temperatura de Boltzmann.